In [1]:
import cobra
from cobra.io import read_sbml_model,load_json_model
from cobra import Model, Reaction, Metabolite
from cobra.medium import minimal_medium
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord

In [2]:
model = load_json_model('../data/GSMM_parameters/LP_WCFS1.json')

# GPR

In [ ]:
# gpr_data=[]
# for gene in  model.genes:
#     for rxn in list(gene.reactions):
#         gpr_data.append({'gene':gene.id,'rxn':rxn.id})    
# gpr_pd = pd.DataFrame(gpr_data)   
# gpr_pd.to_csv('../data/GSMM_parameters/gpr_metnet.csv',index=None)

In [ ]:
gpr_pd = pd.read_csv('../data/GSMM_parameters/gpr_metnet.csv')
uniprot_info = pd.read_csv('../data/Proteomics/uniprotkb_taxonomy_id_220668.tsv',sep='\t')
uniprot_info = uniprot_info.rename({'Gene Names (ordered locus)':'locus_tag'}, axis='columns')
metgene_tags = list( set(list(gpr_pd['gene'])) )
print(len(metgene_tags))
matched_pd = (uniprot_info[uniprot_info.locus_tag.isin(metgene_tags)]).reset_index().drop(['index'],axis=1)
metgene_uids = list(matched_pd.Entry)+['F9UST1']

In [ ]:
# wcfs1_proteins = SeqIO.parse( "../data/Proteomics/wcfs1_prot.fasta", "fasta")
# met_proteins = []
# for seq_record in wcfs1_proteins:
#     if str(seq_record.id) in metgene_uids:
#         if str(seq_record.id) == 'F9UST1':
#             locus_tag = 'lp_0106/lp_0107'
#         else:
#             locus_tag = list(matched_pd[matched_pd['Entry'] == str(seq_record.id)]['locus_tag'])[0]
#         met_proteins.append( SeqRecord(Seq( str(seq_record.seq).replace('*','') ),\
#                                     id = locus_tag+'|'+str(seq_record.id), name="",description="") )
# SeqIO.write( met_proteins, '../data/Genome_HMX2/wcfs1_metabolic_proteins.fasta' ,"fasta")

# Remove genes

# Add reactions

In [3]:
model.reactions.FBA.id = 'FDPA'#avoid confusion with FBA(flux balance analysis)
#fix directions
model.reactions.PGI.lower_bound = 0.0
model.reactions.FDPA.lower_bound = 0.0#FBA
model.reactions.GAPD.lower_bound = 0.0
model.reactions.GALU.lower_bound = 0.0
model.reactions.UDPG4E.lower_bound = 0.0
model.reactions.PGK.lower_bound = 0.0
model.reactions.PGM.lower_bound = 0.0
model.reactions.ENO.lower_bound = 0.0
model.reactions.LDH_L.build_reaction_from_string('h_c + nadh_c + pyr_c --> lac__L_c + nad_c')
model.reactions.LDH_L.lower_bound = 0.0
model.reactions.PFL.lower_bound = 0.0
model.reactions.PTAr.lower_bound = 0.0
model.reactions.ACKr.build_reaction_from_string('actp_c + adp_c --> ac_c + atp_c')
model.reactions.ACKr.lower_bound = 0.0
model.reactions.MAN6PI.build_reaction_from_string('f6p_c --> man6p_c')
model.reactions.MAN6PI.lower_bound = 0.0

# Pseudo-reactions of EPS repeating unit synthesis

In [4]:
gdpmann_c = Metabolite('gdpmann_c','C16H23N5O16P2','GDP-D-mannose',-2,'default')
manx14_c = Metabolite('manx14_c','C84H154O70','Mannose residues of EPS repeating unit',0,'default')
manx14_glcx6_c = Metabolite('manx14_glcx6_c','C120H220O100','Mannose and glucose residues of EPS repeating unit',0,'default')
EPS_c = Metabolite('EPS_c','C126H231O105','EPS repeating unit',0,'default')
EPS_e = Metabolite('EPS_e','C126H231O105','EPS repeating unit',0,'default')
model.add_metabolites([gdpmann_c, manx14_c,manx14_glcx6_c,EPS_c, EPS_e ])

MAN1PT = Reaction('MAN1PT','Mannose-1-phosphate guanylyltransferase (GDP)', 0, 1000)
MANT_EPS = Reaction('MANT_EPS','Mannosyltransferase for EPS formation', 0, 1000)
GLCT_EPS = Reaction('GLCT_EPS','Glucosyltransferase for EPS formation', 0, 1000)
GALT_EPS = Reaction('GALT_EPS','Galactosyltransferase for EPS formation', 0, 1000)
WZX = Reaction('WZX','Wzx flippase-mediated membrane translocation','',0,1000)
EX_EPS_e = Reaction('EX_EPS_e','EX_EPS_e','',-1000,1000)
model.add_reactions([MAN1PT, MANT_EPS, GLCT_EPS,GALT_EPS, WZX,EX_EPS_e ])

model.reactions.MAN1PT.build_reaction_from_string('gdp_c + h_c + man1p_c --> gdpmann_c + pi_c')
model.reactions.MANT_EPS.build_reaction_from_string('gdpmann_c --> 0.07143 manx14_c + gdp_c')
model.reactions.GLCT_EPS.build_reaction_from_string('udpg_c + 0.16667 manx14_c --> 0.16667 manx14_glcx6_c + udp_c')
model.reactions.GALT_EPS.build_reaction_from_string('udpgal_c + manx14_glcx6_c --> EPS_c + udp_c')
model.reactions.WZX.build_reaction_from_string('EPS_c + atp_c + h2o_c --> EPS_e + adp_c + pi_c +h_c' )
model.reactions.EX_EPS_e.build_reaction_from_string('EPS_e <=> ')

# Save the modified model

In [5]:
cobra.io.save_json_model(model, "../data/GSMM_parameters/LP_HMX2.json")